### install pyspark

In [ ]:
#pip install pyspark

### import libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import re 
import pandas as pd
import numpy as np
from time import sleep
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
#import sys

In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

### url link
#### check response = 200

In [4]:
url = "https://www.atptour.com/en/rankings/singles?rankRange=0-100&rankDate=2022-06-27"
headers={'User-Agent': ''}
page = requests.get(url,timeout=15, headers= headers)
print(page)


<Response [200]>


### main ranking table

In [5]:
soup = BeautifulSoup(page.content, "html.parser")
rows = soup.find("div", {"class": "table-rankings-wrapper"}).find_all('tr')

### 1. ranking

In [6]:
ranking = rows[1].find('td', {"class":"rank-cell"}).get_text().strip()
ranking

'1'

### 2. country

In [7]:
country = rows[1].find("div", {"class": "country-item"}).find("img").get('alt')
country

'RUS'

### 3. name

In [8]:
name = rows[1].find("a", {"class": ""}).get_text().strip()
name

'Daniil Medvedev'

### 4. age

In [9]:
age = rows[1].find('td', {"class": "age-cell"}).get_text().strip()
age

'26'

### 5. points

In [10]:
points = rows[1].find('td', {"class":"points-cell"}).get_text().strip()
points

'7,955'

### 6. tournament played

In [12]:
tournaments = rows[1].find('td', {'class':"tourn-cell"}).get_text().strip()
tournaments

'21'

### 7. date

In [13]:
date = soup.find('div', class_= 'dropdown-layout-wrapper rank-detail-filter').find_all('ul')[2].find_all("li")[1].get_text().strip().replace(".", "-", 2)

In [14]:
date

'2022-06-27'

In [ ]:
#nr_rows = len(soup.find('table', class_="mega-table" ).find('tbody').find_all('tr'))
#nr_rows

### first row of most recent date

In [15]:
df = pd.DataFrame([{
    "ranking": ranking,
    "country": country,
    "player": name,
    "age": age,
    "points": points,
    "tournaments": tournaments,
    "date": date
}])

df

,ranking,country,player,age,points,tournaments,date
0,1,RUS,Daniil Medvedev,26,"7,955",21,2022-06-27


### create a list with dates

In [16]:
drop_down=soup.find('div', {'class': 'main-content'}).find('ul',{'class': 'dropdown'})
#first one repeats so removing 0th element
date_list=[]

dates =soup.find('div', class_= 'dropdown-layout-wrapper rank-detail-filter').find_all('ul')[2].find_all("li")

for x in dates[1:]:
    date_list.append(x.get('data-value'))

### grab last 2 years

In [17]:
date_list[:96]

['2022-06-27',
 '2022-06-20',
 '2022-06-13',
 '2022-06-06',
 '2022-05-23',
 '2022-05-16',
 '2022-05-09',
 '2022-05-02',
 '2022-04-25',
 '2022-04-18',
 '2022-04-11',
 '2022-04-04',
 '2022-03-21',
 '2022-03-07',
 '2022-02-28',
 '2022-02-21',
 '2022-02-14',
 '2022-02-07',
 '2022-01-31',
 '2022-01-17',
 '2022-01-10',
 '2022-01-03',
 '2021-12-27',
 '2021-12-20',
 '2021-12-13',
 '2021-12-06',
 '2021-11-29',
 '2021-11-22',
 '2021-11-15',
 '2021-11-08',
 '2021-11-01',
 '2021-10-25',
 '2021-10-18',
 '2021-10-04',
 '2021-09-27',
 '2021-09-20',
 '2021-09-13',
 '2021-08-30',
 '2021-08-23',
 '2021-08-16',
 '2021-08-09',
 '2021-08-02',
 '2021-07-26',
 '2021-07-19',
 '2021-07-12',
 '2021-06-28',
 '2021-06-21',
 '2021-06-14',
 '2021-05-31',
 '2021-05-24',
 '2021-05-17',
 '2021-05-10',
 '2021-05-03',
 '2021-04-26',
 '2021-04-19',
 '2021-04-12',
 '2021-04-05',
 '2021-03-22',
 '2021-03-15',
 '2021-03-08',
 '2021-03-01',
 '2021-02-22',
 '2021-02-08',
 '2021-02-01',
 '2021-01-25',
 '2021-01-18',
 '2021-01-

In [18]:
date

'2022-06-27'

### scrape function

In [19]:
df = pd.DataFrame()

def scrape(date):
    urlpattern = "https://www.atptour.com/en/rankings/singles?rankRange=0-100&rankDate={}"
    url = urlpattern.format(date)
    headers={'User-Agent': ''}
    response = requests.get(url,timeout=15, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        rows = soup.find("div", {"class": "table-rankings-wrapper"}).find_all('tr') 
        lst = []
        for row in rows[1:]:
            
            #Ranking
            try:
                ranking = row.find("td", {"class": "rank-cell"}).get_text().strip()
            except:
                ranking = np.nan 
            
            #Country
            try:
                country = row.find("div", {"class": "country-item"}).find("img")['alt']
            except:
                country = np.nan
            
            #Player
            try:
                player = row.find("td", {"class": "player-cell"}).get_text().strip()
            except:
                player = np.nan
            
            #Age
            try:
                age = row.find("td", {"class": "age-cell"}).get_text().strip()
            except:
                age = np.nan
            
            #Points
            try:
                points = row.find("a", {"ga-label": "rankings-breakdown"}).get_text().strip()
            except:
                points = np.nan
            
            #Tournaments
            try:
                tournaments = row.find("td", {"class": "tourn-cell"}).get_text().strip()
            except:
                tournaments = np.nan
            
            
            temp = {
                "ranking": ranking,
                "country": country,
                "player": player,
                "age": age,
                "points": points,
                "tournaments": tournaments,
                "date": date
            }
            lst.append(temp)
            
#             df = df.append(temp, ignore_index=True)
                
    else:
        print('Scraper is down!')
            
    return pd.DataFrame(lst)

### scrape top100 of the most recent date (2022-06-27)

In [20]:
df = scrape(date)

In [21]:
df

,ranking,country,player,age,points,tournaments,date
0,1,RUS,Daniil Medvedev,26,"7,955",21,2022-06-27
1,2,GER,Alexander Zverev,25,"7,030",19,2022-06-27
2,3,SRB,Novak Djokovic,35,"6,770",12,2022-06-27
3,4,ESP,Rafael Nadal,36,"6,525",10,2022-06-27
4,5,GRE,Stefanos Tsitsipas,23,"5,150",25,2022-06-27
...,...,...,...,...,...,...,...
95,96,SUI,Henri Laaksonen,30,606,25,2022-06-27
96,97,SUI,Roger Federer,40,600,3,2022-06-27
97,98,ESP,Carlos Taberner,24,585,35,2022-06-27
98,99,AUS,Jason Kubler,29,580,27,2022-06-27


### scrape top 100 for dates:2019-2022
#### this is the final dataframe

In [22]:
df_final = pd.DataFrame(columns=["ranking", "country", "player", "points", "tournaments", "date"])
dates_df = df['date'].unique()

for date in tqdm(date_list[:96]):
    if date not in dates_df:
        temp_df = scrape(date)
        df_final = pd.concat([df_final, temp_df],ignore_index=True)
    else:
        pass

  0%|          | 0/96 [00:00<?, ?it/s]

In [23]:
df_final

,ranking,country,player,points,tournaments,date,age
0,1,RUS,Daniil Medvedev,"8,160",21,2022-06-20,26
1,2,GER,Alexander Zverev,"7,030",19,2022-06-20,25
2,3,SRB,Novak Djokovic,"6,770",12,2022-06-20,35
3,4,ESP,Rafael Nadal,"6,525",10,2022-06-20,36
4,5,NOR,Casper Ruud,"5,050",28,2022-06-20,23
...,...,...,...,...,...,...,...
9495,96,ITA,Salvatore Caruso,586,28,2019-12-30,27
9496,97,AUS,Alexei Popyrin,585,25,2019-12-30,20
9497,98,BLR,Egor Gerasimov,581,23,2019-12-30,27
9498,99,RSA,Lloyd Harris,576,24,2019-12-30,22


### extract renkings to csv file

In [ ]:
df_final.to_csv(r'C:\Users\sokin\Desktop\atp-rankings.csv', index=False)

### PySpark

In [24]:
spark = SparkSession.builder.appName('trg').getOrCreate()

In [25]:
spark

### let's check the dataframe in pyspark

In [26]:
df_spark = spark.read.csv('atp-rankings.csv')

In [27]:
df_spark = spark.read.option('header', 'true').csv('atp-rankings.csv')
df_spark.show()

+-------+-------+--------------------+------+-----------+----------+---+
|ranking|country|              player|points|tournaments|      date|age|
+-------+-------+--------------------+------+-----------+----------+---+
|      1|    RUS|     Daniil Medvedev| 8,160|         21|2022-06-20| 26|
|      2|    GER|    Alexander Zverev| 7,030|         19|2022-06-20| 25|
|      3|    SRB|      Novak Djokovic| 6,770|         12|2022-06-20| 35|
|      4|    ESP|        Rafael Nadal| 6,525|         10|2022-06-20| 36|
|      5|    NOR|         Casper Ruud| 5,050|         28|2022-06-20| 23|
|      6|    GRE|  Stefanos Tsitsipas| 4,945|         24|2022-06-20| 23|
|      7|    ESP|      Carlos Alcaraz| 4,893|         20|2022-06-20| 19|
|      8|    RUS|       Andrey Rublev| 3,870|         26|2022-06-20| 24|
|      9|    CAN|Felix Auger-Alias...| 3,760|         25|2022-06-20| 21|
|     10|    POL|      Hubert Hurkacz| 3,738|         25|2022-06-20| 25|
|     11|    ITA|   Matteo Berrettini| 3,480|      

### some data manipulation

In [29]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.types import IntegerType

In [30]:
df_spark = df_spark.withColumn('points', regexp_replace('points', ',', ''))
df_spark = df_spark.withColumn("points",df_spark["points"].cast(IntegerType()))
df_spark = df_spark.withColumn("tournaments",df_spark["tournaments"].cast(IntegerType()))
df_spark = df_spark.withColumn("age",df_spark["age"].cast(IntegerType())) 
df_spark = df_spark.withColumn("ranking",df_spark["ranking"].cast(IntegerType())) 

df_spark.select('points', 'tournaments', 'age').dtypes
df_spark.show()

+-------+-------+--------------------+------+-----------+----------+---+
|ranking|country|              player|points|tournaments|      date|age|
+-------+-------+--------------------+------+-----------+----------+---+
|      1|    RUS|     Daniil Medvedev|  8160|         21|2022-06-20| 26|
|      2|    GER|    Alexander Zverev|  7030|         19|2022-06-20| 25|
|      3|    SRB|      Novak Djokovic|  6770|         12|2022-06-20| 35|
|      4|    ESP|        Rafael Nadal|  6525|         10|2022-06-20| 36|
|      5|    NOR|         Casper Ruud|  5050|         28|2022-06-20| 23|
|      6|    GRE|  Stefanos Tsitsipas|  4945|         24|2022-06-20| 23|
|      7|    ESP|      Carlos Alcaraz|  4893|         20|2022-06-20| 19|
|      8|    RUS|       Andrey Rublev|  3870|         26|2022-06-20| 24|
|      9|    CAN|Felix Auger-Alias...|  3760|         25|2022-06-20| 21|
|     10|    POL|      Hubert Hurkacz|  3738|         25|2022-06-20| 25|
|     11|    ITA|   Matteo Berrettini|  3480|      

In [31]:
df_spark.select('ranking', 'points', 'tournaments', "age").dtypes

[('ranking', 'int'), ('points', 'int'), ('tournaments', 'int'), ('age', 'int')]

## The goal is to create a streaming process that will read each input file(a dataframe per date) for every unique date. A transformation will be implemented in order to find the sum of points for every country


###  sum of points for each country per date

In [33]:
df_spark.groupby('date', 'country').sum('points').sort('date',ascending=False).show(100)
#df_spark.groupby('date', 'country').orderBy(col("date").desc,col("sum(points)").desc)

+----------+-------+-----------+
|      date|country|sum(points)|
+----------+-------+-----------+
|2022-06-20|    FRA|       7423|
|2022-06-20|    NED|       2477|
|2022-06-20|    SWE|        659|
|2022-06-20|    CZE|       1615|
|2022-06-20|    BRA|        622|
|2022-06-20|    NOR|       5050|
|2022-06-20|    BLR|       1082|
|2022-06-20|    TPE|        626|
|2022-06-20|    FIN|       1015|
|2022-06-20|    CAN|       6053|
|2022-06-20|    HUN|        932|
|2022-06-20|    BUL|       1785|
|2022-06-20|    ESP|      21829|
|2022-06-20|    RSA|       1015|
|2022-06-20|    GRE|       4945|
|2022-06-20|    ARG|       6800|
|2022-06-20|    GER|       9693|
|2022-06-20|    BEL|        930|
|2022-06-20|    KOR|        772|
|2022-06-20|    CRO|       2220|
|2022-06-20|    BOL|        729|
|2022-06-20|    POR|        918|
|2022-06-20|    USA|      17105|
|2022-06-20|    SVK|       1002|
|2022-06-20|    GBR|       5396|
|2022-06-20|    SUI|       1206|
|2022-06-20|    POL|       4373|
|2022-06-2

### the transformation I want to implement(for this example I choose date=2022-06-20)

In [34]:
df_spark.where(df_spark.date == "2022-06-20").groupby('date', 'country').sum('points').sort('sum(points)',ascending=False).show()

+----------+-------+-----------+
|      date|country|sum(points)|
+----------+-------+-----------+
|2022-06-20|    ESP|      21829|
|2022-06-20|    USA|      17105|
|2022-06-20|    RUS|      14841|
|2022-06-20|    SRB|      11260|
|2022-06-20|    GER|       9693|
|2022-06-20|    ITA|       9578|
|2022-06-20|    FRA|       7423|
|2022-06-20|    AUS|       6941|
|2022-06-20|    ARG|       6800|
|2022-06-20|    CAN|       6053|
|2022-06-20|    GBR|       5396|
|2022-06-20|    NOR|       5050|
|2022-06-20|    GRE|       4945|
|2022-06-20|    POL|       4373|
|2022-06-20|    NED|       2477|
|2022-06-20|    CRO|       2220|
|2022-06-20|    CHI|       1793|
|2022-06-20|    BUL|       1785|
|2022-06-20|    CZE|       1615|
|2022-06-20|    GEO|       1473|
+----------+-------+-----------+
only showing top 20 rows



In [36]:
steps = df_spark.select("date").distinct().collect()
for step in steps[:]:
    _df_spark = df_spark.where(df_spark.date == step[0])
    _df_spark.coalesce(1).write.mode("append").option("header","true").csv('\trg_data')

Py4JJavaError: An error occurred while calling o163.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:638)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:278)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)
	a

In [ ]:
step = spark.read.csv('filepath', header=True, inferSchema=True)

step.groupby('date', 'country').sum('points').sort('date',ascending=False).show()

In [ ]:
dataSchema = step.schema